This is the coding part for data retrieval and data visualisation

<b>Library import</b>

In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
import json

In [4]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

In [5]:
import requests
from pandas.io.json import json_normalize 

In [6]:
from sklearn.cluster import KMeans

In [7]:
import folium

In [8]:
CLIENT_ID = 'FIKYPXQAX5S0EILPMQLSCIDIWEPLPFMZSWZPFLCUFKIR4YWC'
CLIENT_SECRET = '2UOTTIJCZ1FFA5GWIJTGTN4UUH5E5GBNZVDGWTVXTJFZ32LG'
VERSION = '20180605'

<b>Company Location</b>

1.LOCATING

In [9]:
company=input('Which company are you going to intern for:')

Which company are you going to intern for: KPMG


In [10]:
url_c = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    38.8976797, 
    -77.0365191, 
    company)

In [11]:
results = requests.get(url_c).json()

In [12]:
res=results['response']['venues']
name=[]
lat=[]
lng=[]
add=[]
zipcd=[]
state=[]
for n in res:
    m=n['location']
    if m['state']=='D.C.' and m['city']=='Washington':
        name.append(n['name'])
        lat.append(m['lat'])
        lng.append(m['lng'])
        add.append(m['formattedAddress'])
        zipcd.append(m['postalCode'])
cpn=pd.DataFrame({'name':name, 'zip':zipcd, 'lat':lat, 'lng':lng, 'add':add})
lat_=lat[0]
lng_=lng[0]

<b>Company Surroundings</b>

2.EXPLORING

In [13]:
radius=float(input('How far away from your company can you stand for commuting (in meters):'))

How far away from your company can you stand for commuting (in meters): 3000


In [14]:
number=float(input('How many venues in the neighbourhood do you want to check at a glance (100 max.):'))

How many venues in the neighbourhood do you want to check at a glance (100 max.): 200


In [15]:
url_s = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat_, 
    lng_, 
    radius,
    number)

In [16]:
results = requests.get(url_s).json()

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.location.postalCode', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

3.ANALYSING

In [19]:
onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

onehot['postalCode'] = nearby_venues['postalCode'] 

fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

In [20]:
company_grouped = onehot.groupby('postalCode').mean().reset_index()

In [21]:
num_top_venues = 5

for hood in company_grouped['postalCode']:
    print("----"+hood+"----")
    temp = company_grouped[company_grouped['postalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----20001----
                venue  freq
0      Ice Cream Shop   0.2
1      History Museum   0.2
2  Salon / Barbershop   0.1
3              Bakery   0.1
4  Mexican Restaurant   0.1


----20002----
                       venue  freq
0         Spanish Restaurant   1.0
1        American Restaurant   0.0
2                Pizza Place   0.0
3         Italian Restaurant   0.0
4  Latin American Restaurant   0.0


----20004----
                   venue  freq
0  Performing Arts Venue  0.17
1                 Bakery  0.17
2                  Hotel  0.17
3                Theater  0.17
4             Taco Place  0.17


----20005----
                 venue  freq
0                Hotel  0.17
1  American Restaurant  0.08
2   Salon / Barbershop  0.08
3   Seafood Restaurant  0.08
4           Steakhouse  0.08


----20006----
                 venue  freq
0                Hotel  0.38
1  American Restaurant  0.12
2          Coffee Shop  0.12
3       Sandwich Place  0.12
4            Hotel Bar  0.12


----2000

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalCode_venues_sorted = pd.DataFrame(columns=columns)
postalCode_venues_sorted['postalCode'] = company_grouped['postalCode']

for ind in np.arange(company_grouped.shape[0]):
    postalCode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(company_grouped.iloc[ind, :], num_top_venues)

4.CLUSTERING

In [24]:
kclusters = 6

company_grouped_clustering = company_grouped.drop('postalCode', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(company_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 4, 0, 2, 1], dtype=int32)

In [25]:
# add clustering labels
postalCode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

company_merged = nearby_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
company_merged = company_merged.join(postalCode_venues_sorted.set_index('postalCode'), on='postalCode')

company_merged.head() # check the last columns!

,postalCode,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20036,Wawa,Convenience Store,38.904303,-77.043356,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
1,20006,Compass Coffee,Coffee Shop,38.901041,-77.041680,0.0,Hotel,Coffee Shop,Indian Restaurant,Hotel Bar,Sandwich Place,American Restaurant,Bookstore,French Restaurant,Art Museum,History Museum
2,20036,DC Pizza,Pizza Place,38.904038,-77.043186,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
3,20036,Boqueria,Spanish Restaurant,38.905921,-77.043140,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
4,20006,Founding Farmers,American Restaurant,38.900573,-77.044519,0.0,Hotel,Coffee Shop,Indian Restaurant,Hotel Bar,Sandwich Place,American Restaurant,Bookstore,French Restaurant,Art Museum,History Museum


<b>Housing Address</b>

In [26]:
df=pd.read_csv('https://opendata.arcgis.com/datasets/aa514416aaf74fdc94748f1e56e7cc8a_0.csv')
data={'ADDRESS_ID':df['ADDRESS_ID'],'LATITUDE':df['LATITUDE'], 'LONGITUDE':df['LONGITUDE'], 'STATUS':df['STATUS'], 'RES_TYPE':df['RES_TYPE'],'STNAME':df['STNAME'],'ASSESSMENT_NBHD':df['ASSESSMENT_NBHD'],'ZIPCODE':df['ZIPCODE']}
df1=pd.DataFrame(data)

/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10,18,19,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
pc=nearby_venues.drop_duplicates('postalCode')

In [43]:
frame=[]
for n in pc['postalCode']:
    for m in df1['ZIPCODE']:
        try:
            if m==float(n):
                df2=df1.loc[df1['ZIPCODE']==m]
                frame.append(df2)
                break
        except:
            pass
dfx=pd.concat(frame)
if len(dfx)>= 1000:
    df2=dfx[0:1000]
else:
    df2=dfx[0:len(dfx)-1]

In [44]:
df2.shape

(1000, 8)

In [45]:
# create map
map_clusters = folium.Map(location=[lat_, lng_], zoom_start=12.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(company_merged['lat'], company_merged['lng'], company_merged['postalCode'], company_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[0:6],
        fill=True,
        fill_color=rainbow[0:6],
        fill_opacity=0.7).add_to(map_clusters)

for name, lat, lng, add in zip(cpn['name'], cpn['lat'], cpn['lng'], cpn['add']):
    label = '{}, {}'.format(company, add)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lng],
        popup=label).add_to(map_clusters)  

for lat, lng, STATUS, RES_TYPE, STNAME, ASSESSMENT_NBHD in zip(df2['LATITUDE'], df2['LONGITUDE'], df2['STATUS'], df2['RES_TYPE'],df2['STNAME'],df2['ASSESSMENT_NBHD']):
    if RES_TYPE!='NON RESIDENTIAL' and STATUS=='ACTIVE':
        label = '{}, {}'.format(RES_TYPE, STATUS, STNAME, ASSESSMENT_NBHD)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_clusters)  
    
map_clusters

5.Examining

In [46]:
company_merged.loc[company_merged['Cluster Labels'] == 0, company_merged.columns[[1] + list(range(5, company_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wawa,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
1,Compass Coffee,0.0,Hotel,Coffee Shop,Indian Restaurant,Hotel Bar,Sandwich Place,American Restaurant,Bookstore,French Restaurant,Art Museum,History Museum
2,DC Pizza,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
3,Boqueria,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
4,Founding Farmers,0.0,Hotel,Coffee Shop,Indian Restaurant,Hotel Bar,Sandwich Place,American Restaurant,Bookstore,French Restaurant,Art Museum,History Museum
5,Bub and Pop's,0.0,Bookstore,Hotel,Pizza Place,Park,Convenience Store,Comic Shop,Italian Restaurant,Greek Restaurant,Farmers Market,Salad Place
6,AKA White House,0.0,Hotel,Coffee Shop,Indian Restaurant,Hotel Bar,Sandwich Place,American Restaurant,Bookstore,French Restaurant,Art Museum,History Museum
7,The Bombay Club,0.0,Hotel,Coffee Shop,Indian Restaurant,Hotel Bar,Sandwich Place,American Restaurant,Bookstore,French Restaurant,Art Museum,History Museum
8,Renwick Gallery,0.0,Park,Art Museum,Government Building,Wine Bar,Falafel Restaurant,History Museum,Gym,Grocery Store,Greek Restaurant,Gastropub
9,BLT Steak,0.0,Hotel,Steakhouse,Nightlife Spot,Cycle Studio,Pizza Place,Salon / Barbershop,Coffee Shop,Seafood Restaurant,American Restaurant,Beer Bar


In [47]:
company_merged.loc[company_merged['Cluster Labels'] == 1, company_merged.columns[[1] + list(range(5, company_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,Vietnam Veterans Memorial,1.0,Monument / Landmark,Wine Bar,Hotel Bar,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden
82,World War II Memorial,1.0,Monument / Landmark,Wine Bar,Hotel Bar,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden
90,Lincoln Memorial Reflecting Pool,1.0,Monument / Landmark,Wine Bar,Hotel Bar,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden
97,Washington Monument,1.0,Monument / Landmark,Wine Bar,Hotel Bar,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden


In [48]:
company_merged.loc[company_merged['Cluster Labels'] == 2, company_merged.columns[[1] + list(range(5, company_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Pearl Dive Oyster Palace,2.0,Seafood Restaurant,Wine Bar,Ethiopian Restaurant,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden


In [49]:
company_merged.loc[company_merged['Cluster Labels'] == 3, company_merged.columns[[1] + list(range(5, company_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Estadio,3.0,Spanish Restaurant,Wine Bar,Ethiopian Restaurant,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden


In [50]:
company_merged.loc[company_merged['Cluster Labels'] == 4, company_merged.columns[[1] + list(range(5, company_merged.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Rock Creek Running Trail,4.0,Trail,Wine Bar,Ethiopian Restaurant,History Museum,Gym,Grocery Store,Greek Restaurant,Government Building,Gastropub,Garden


In [ ]:
company_merged.loc[company_merged['Cluster Labels'] == 5, company_merged.columns[[1] + list(range(5, company_merged.shape[1]))]]